In [2]:
# [2회차 기출유형 작업형2] : E-Commerce Shipping Data

# 시험환경 세팅 (코드 변경 X)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

def exam_data_load(df, target, id_name="", null_name=""):
    if id_name == "":
        df = df.reset_index().rename(columns={"index": "id"})
        id_name = 'id'
    else:
        id_name = id_name
    
    if null_name != "":
        df[df == null_name] = np.nan
    
    X_train, X_test = train_test_split(df, test_size=0.2, random_state=2021)
    
    y_train = X_train[[id_name, target]]
    X_train = X_train.drop(columns=[target])

    
    y_test = X_test[[id_name, target]]
    X_test = X_test.drop(columns=[target])
    return X_train, X_test, y_train, y_test 
    
df = pd.read_csv("./data/Train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((8799, 11), (2200, 11), (8799, 2), (2200, 2))

In [75]:
# 풀이

import pandas as pd

df = pd.read_csv("./data/Train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

# print(X_train.head())
# print(X_train.info())
# print(X_train.isnull().sum()) # 0
# (8799, 11) (2200, 11) (8799, 2) (2200, 2)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 일단 라벨 인코더로 object -> int 변환하고
# Cost_of_the_Product, Weight_in_gms 스케일 변환하고
# 확률 아니니까 predict_proba 말고 predict로 나눠야 될거 같고
# 분류 문제니까 Classifier 말고 Regression 써야 될거 같고
# roc_auc_score 써서 모델 성능 평가해야되고 (metrics)

from sklearn.preprocessing import LabelEncoder
# import sklearn
# dir(sklearn)

df = pd.concat([X_train, X_test])
# print(df.shape)

# print(df.describe(include='object').columns)
cols = df.describe(include='object').columns

le = LabelEncoder()
# le.fit_transform(df)
for col in cols:
    df[col] = le.fit_transform(df[col])
# print(df.head())

# 이거 스케일까지 하고 나누자
# X_train = df[:X_train.shape[0]]
# X_test = df[X_train.shape[0]:]
# print(X_train.head())
# print(X_train.shape, X_test.shape)

from sklearn.preprocessing import minmax_scale
# import sklearn.preprocessing
# dir(sklearn.preprocessing)

# print(X_train.head())
# minmax_scale(X_train['Cost_of_the_Product'])
df['Cost_of_the_Product'] = minmax_scale(df['Cost_of_the_Product'])
df['Weight_in_gms'] = minmax_scale(df['Weight_in_gms'])

X_train = df[:X_train.shape[0]]
X_test = df[X_train.shape[0]:]

# print(X_train.head())

from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
# print(dir(sklearn.tree)) # DecisionTreeRegressor
# print(dir(sklearn.ensemble)) # RandomForestRegressor

model = DecisionTreeRegressor()
model.fit(X_train, y=y_train)
pred = model.predict(X_test)
print(roc_auc_score(y_test.set_index('ID',drop=True), pred[:,1]))
print(pred)

model = RandomForestRegressor()
model.fit(X_train, y_train)
pred = model.predict(X_test)
print(roc_auc_score(y_test.set_index('ID',drop=True), pred[:,1]))
# print(pred[:,1])
print(pred)

submit = pd.DataFrame()
submit['ID'] = y_test['ID']
# print(y_test.head())
submit['Reached.on.Time_Y.N'] = pred[:,1]

submit.to_csv('E_Com_1.csv', index=False)
# print(help(submit.to_csv))

submit = pd.read_csv('E_Com_1.csv')
print(submit.head())

# 씁... 확률로 나왔네?
# 아 그리고 random_state 안넣었네
# pred[:,1]

0.6329824489123269
[[4.731e+03 0.000e+00]
 [2.042e+03 1.000e+00]
 [5.118e+03 1.000e+00]
 ...
 [9.752e+03 0.000e+00]
 [3.746e+03 0.000e+00]
 [4.413e+03 0.000e+00]]
0.7279491286255768
[[4.73196e+03 2.30000e-01]
 [2.03913e+03 1.00000e+00]
 [5.11499e+03 8.10000e-01]
 ...
 [9.75266e+03 5.40000e-01]
 [3.75090e+03 2.30000e-01]
 [4.41462e+03 3.30000e-01]]
     ID  Reached.on.Time_Y.N
0  4733                 0.23
1  2040                 1.00
2  5114                 0.81
3  2361                 1.00
4  5996                 0.63


In [99]:
# 해답

df = pd.read_csv('./data/Train.csv')
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

# print(y_train['Reached.on.Time_Y.N'].value_counts())
# print(X_train[['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender']].nunique())

X_train = X_train.drop(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], axis=1)
X_test = X_test.drop(['Warehouse_block', 'Mode_of_Shipment', 'Product_importance', 'Gender'], axis=1)
print(X_train.shape, X_test.shape)

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

X_train_id = X_train.pop('ID')
X_test_id = X_test.pop('ID')

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train['Reached.on.Time_Y.N'], test_size=0.2, random_state=2021)

model = LogisticRegression()
model.fit(X_tr, y_tr)
print(round(model.score(X_val, y_val)*100, 2))
pred = model.predict(X_val)
print(pred)
print(roc_auc_score(y_val, pred))
# print(X_tr.shape, X_val.shape, y_tr.shape, y_val.shape)
print(X_val.head())
print(y_val.head())


# model = KNeighborsClassifier()
# model.fit(X_tr, y_tr)
# print(round(model.score(X_val, y_val)*100, 2))
# pred = model.predict(X_val)
# print(pred)

# model = SVC()
# model.fit(X_tr, y_tr)
# print(round(model.score(X_val, y_val)*100, 2))
# pred = model.predict(X_val)
# print(pred)

# model = DecisionTreeClassifier()
# model.fit(X_tr, y_tr)
# print(round(model.score(X_val, y_val)*100, 2))
# pred = model.predict(X_val)
# print(pred)

# model = RandomForestClassifier()
# model.fit(X_tr, y_tr)
# print(round(model.score(X_val, y_val)*100, 2))
# pred = model.predict(X_val)
# print(pred)

# model = XGBClassifier()
# model.fit(X_tr, y_tr)
# print(round(model.score(X_val, y_val)*100, 2))
# pred = model.predict(X_val)
# print(pred)

(8799, 7) (2200, 7)
63.86
[1 0 0 ... 0 1 1]
0.6294943760498395
      Customer_care_calls  Customer_rating  Cost_of_the_Product  \
9248                    4                2                  231   
5567                    4                2                  245   
8091                    3                4                  184   
1674                    3                3                  267   
4437                    3                4                  176   

      Prior_purchases  Discount_offered  Weight_in_gms  
9248                4                 9           5194  
5567                2                 3           4000  
8091                3                 2           4875  
1674                2                18           2435  
4437                2                10           4920  
9248    0
5567    0
8091    0
1674    1
4437    0
Name: Reached.on.Time_Y.N, dtype: int64


In [115]:
# 풀이 수정

import pandas as pd

df = pd.read_csv("./data/Train.csv")
X_train, X_test, y_train, y_test = exam_data_load(df, target='Reached.on.Time_Y.N', id_name='ID')

# print(X_train.head())
# print(X_train.info())
# print(X_train.isnull().sum()) # 0
# (8799, 11) (2200, 11) (8799, 2) (2200, 2)
# print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

# 일단 라벨 인코더로 object -> int 변환하고
# Cost_of_the_Product, Weight_in_gms 스케일 변환하고
# 확률 아니니까 predict_proba 말고 predict로 나눠야 될거 같고
# 분류 문제니까 Classifier 말고 Regression 써야 될거 같고
# roc_auc_score 써서 모델 성능 평가해야되고 (metrics)

from sklearn.preprocessing import LabelEncoder
# import sklearn
# dir(sklearn)

df = pd.concat([X_train, X_test])
# print(df.shape)

# print(df.describe(include='object').columns)
cols = df.describe(include='object').columns

le = LabelEncoder()
# le.fit_transform(df)
for col in cols:
    df[col] = le.fit_transform(df[col])
# print(df.head())

# 이거 스케일까지 하고 나누자
# X_train = df[:X_train.shape[0]]
# X_test = df[X_train.shape[0]:]
# print(X_train.head())
# print(X_train.shape, X_test.shape)

from sklearn.preprocessing import minmax_scale
# import sklearn.preprocessing
# dir(sklearn.preprocessing)

# print(X_train.head())
# minmax_scale(X_train['Cost_of_the_Product'])
df['Cost_of_the_Product'] = minmax_scale(df['Cost_of_the_Product'])
df['Weight_in_gms'] = minmax_scale(df['Weight_in_gms'])

X_train = df[:X_train.shape[0]]
X_test = df[X_train.shape[0]:]

# print(X_train.head())

from sklearn.metrics import roc_auc_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
# print(dir(sklearn.tree)) # DecisionTreeRegressor
# print(dir(sklearn.ensemble)) # RandomForestRegressor

X_train_id = X_train.pop('ID')
X_test_id = X_test.pop('ID')

from sklearn.model_selection import train_test_split
X_tr, X_val, y_tr, y_val = train_test_split(X_train, y_train['Reached.on.Time_Y.N'], test_size=0.2, random_state=2021)


model = DecisionTreeClassifier()
model.fit(X_tr, y_tr)
print(round(model.score(X_val, y_val)*100, 2))

pred = model.predict(X_val)
print(roc_auc_score(y_val, pred))
# print(y_val)
# print(pred)
# print(X_test.shape, y_test.shape)
# print(X_test.head())
# print(y_test.head())

# print(y_test.shape)
# print(pred[:,1].shape)
# print(y_test.set_index('ID', drop=True).head())
# print(y_train.head())
# print(y_test.head())

model = RandomForestClassifier()
model.fit(X_tr, y_tr)
print(round(model.score(X_val, y_val)*100, 2))

pred = model.predict(X_val)
print(roc_auc_score(y_val, pred))
# # print(pred[:,1])
# print(pred)

model = XGBClassifier()
model.fit(X_tr, y_tr)
print(round(model.score(X_val, y_val)*100, 2))

pred = model.predict(X_val)
print(roc_auc_score(y_val, pred))


print('='*20)
model = XGBClassifier()
# print(y_train.head())
model.fit(X_train, y_train['Reached.on.Time_Y.N'])
print(round(model.score(X_test, y_test['Reached.on.Time_Y.N'])*100, 2))

pred = model.predict(X_test)
print(roc_auc_score(y_test['Reached.on.Time_Y.N'], pred))

submit = pd.DataFrame()
submit['ID'] = X_test_id
# print(y_test.head())
submit['Reached.on.Time_Y.N'] = pred

submit.to_csv('E_Com_2.csv', index=False)
# print(help(submit.to_csv))

submit = pd.read_csv('E_Com_2.csv')
print(submit.head())

# 씁... 확률로 나왔네?
# 아 그리고 random_state 안넣었네, id도 빼야됨

63.12
0.6147142310391019
64.49
0.6490054432183028
66.7
0.6687610299589631
64.0
0.6463105636123928
     ID  Reached.on.Time_Y.N
0  4733                    1
1  2040                    1
2  5114                    1
3  2361                    1
4  5996                    0
